In [1]:
import os

print("Mempersiapkan lingkungan dan membuat folder di penyimpanan sementara...")

os.makedirs('/content/minecraft/worlds', exist_ok=True)
os.makedirs('/content/minecraft/behavior_packs', exist_ok=True)
os.makedirs('/content/minecraft/resource_packs', exist_ok=True)

os.chdir('/content/minecraft')

print(f"Folder kerja sementara dibuat di: {os.getcwd()}")
print("💡 Semua file akan dibuat di penyimpanan sementara dan diarsipkan sebagai NewWorld.zip.")
print("💡 File zip akan tersedia untuk download manual dari panel file Colab.")

Mempersiapkan lingkungan dan membuat folder di penyimpanan sementara...
Folder kerja sementara dibuat di: /content/minecraft
💡 Semua file akan dibuat di penyimpanan sementara dan diarsipkan sebagai NewWorld.zip.
💡 File zip akan tersedia untuk download manual dari panel file Colab.


In [2]:
print("Memulai proses pengunduhan...")

!pip install gdown --quiet

print("\n[1/2] Mengunduh mod 'Actions and Stuff 1.2 BY RAFLYMC.mcpack'...")
!gdown --id 1rlJ3g9T6oTB8BvP9wUWn-vprECIuCNuD -O "Actions and Stuff 1.2 BY RAFLYMC.mcpack"

print("\n[2/2] Mengunduh file server resmi 'bedrock-server-1.21.93.1.zip'...")
!wget -q -O bedrock-server.zip --show-progress "https://www.minecraft.net/bedrockdedicatedserver/bin-linux/bedrock-server-1.21.93.1.zip"

print("\nSemua file berhasil diunduh ke penyimpanan sementara.")

Memulai proses pengunduhan...

[1/2] Mengunduh mod 'Actions and Stuff 1.2 BY RAFLYMC.mcpack'...
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1rlJ3g9T6oTB8BvP9wUWn-vprECIuCNuD
From (redirected): https://drive.google.com/uc?id=1rlJ3g9T6oTB8BvP9wUWn-vprECIuCNuD&confirm=t&uuid=29179447-4b83-4509-958d-e4668971ec2a
To: /content/minecraft/Actions and Stuff 1.2 BY RAFLYMC.mcpack
100% 32.3M/32.3M [00:00<00:00, 60.8MB/s]

[2/2] Mengunduh file server resmi 'bedrock-server-1.21.93.1.zip'...
bedrock-server.zip  100%[===================>]  65.29M   102MB/s    in 0.6s    

Semua file berhasil diunduh ke penyimpanan sementara.


In [3]:
import zipfile
import shutil

print("Mengekstrak file server di penyimpanan sementara...")
!unzip -q -o bedrock-server.zip

mod_filename = "Actions and Stuff 1.2 BY RAFLYMC.mcpack"
print(f"\nMemproses dan mengekstrak file mod '{mod_filename}'...")

mod_target_folder_name = "ActionsAndStuff_ByRaflyMC"
behavior_pack_path = os.path.join('./behavior_packs', mod_target_folder_name)
resource_pack_path = os.path.join('./resource_packs', mod_target_folder_name)

os.makedirs(behavior_pack_path, exist_ok=True)

with zipfile.ZipFile(mod_filename, 'r') as zip_ref:
    zip_ref.extractall(behavior_pack_path)

print(f"✅ Mod 'Actions and Stuff' berhasil diekstrak sebagai Behavior Pack ke: {behavior_pack_path}")

if os.path.exists(resource_pack_path):
    shutil.rmtree(resource_pack_path)
shutil.copytree(behavior_pack_path, resource_pack_path)

print(f"✅ Mod 'Actions and Stuff' berhasil disalin sebagai Resource Pack ke: {resource_pack_path}")

print("\n🔍 Debug - Struktur folder behavior_packs:")
!ls -R ./behavior_packs/ActionsAndStuff_ByRaflyMC | head -n 15

print("\n🔍 Debug - Struktur folder resource_packs:")
!ls -R ./resource_packs/ActionsAndStuff_ByRaflyMC | head -n 15

print("\n🧹 Membersihkan file arsip...")
try:
    os.remove("bedrock-server.zip")
    os.remove(mod_filename)
except FileNotFoundError:
    print("   File arsip sudah dihapus atau tidak ditemukan.")

print("\n✅ Proses ekstraksi selesai!")

Mengekstrak file server di penyimpanan sementara...

Memproses dan mengekstrak file mod 'Actions and Stuff 1.2 BY RAFLYMC.mcpack'...
✅ Mod 'Actions and Stuff' berhasil diekstrak sebagai Behavior Pack ke: ./behavior_packs/ActionsAndStuff_ByRaflyMC
✅ Mod 'Actions and Stuff' berhasil disalin sebagai Resource Pack ke: ./resource_packs/ActionsAndStuff_ByRaflyMC

🔍 Debug - Struktur folder behavior_packs:
./behavior_packs/ActionsAndStuff_ByRaflyMC:
animation_controllers
animations
attachables
entity
manifest.json
materials
models
pack_icon.png
particles
render_controllers
sounds
sounds.json
splashes.json
subpacks

🔍 Debug - Struktur folder resource_packs:
./resource_packs/ActionsAndStuff_ByRaflyMC:
animation_controllers
animations
attachables
entity
manifest.json
materials
models
pack_icon.png
particles
render_controllers
sounds
sounds.json
splashes.json
subpacks

🧹 Membersihkan file arsip...

✅ Proses ekstraksi selesai!


In [4]:
import json

print("Mengkonfigurasi behavior pack dan resource pack untuk dunia...")

behavior_pack_folder = '/content/minecraft/behavior_packs'
resource_pack_folder = '/content/minecraft/resource_packs'
valid_packs_file = '/content/minecraft/valid_known_packs.json'
world_name = "NewWorld"
world_folder = f'/content/minecraft/worlds/{world_name}'

mod_folder_name_to_find = "ActionsAndStuff_ByRaflyMC"

if os.path.exists(world_folder):
    print(f"\n🧹 Menghapus dunia lama '{world_name}' untuk memastikan mod aktif...")
    shutil.rmtree(world_folder)
os.makedirs(world_folder, exist_ok=True)

valid_packs_config = []
world_behavior_packs = []
world_resource_packs = []

def process_packs(pack_directory, pack_type):
    print(f"\n📦 Mencari {pack_type} packs...")
    for root, dirs, files in os.walk(pack_directory):
        if 'manifest.json' in files:
            manifest_path = os.path.join(root, 'manifest.json')
            try:
                with open(manifest_path, 'r') as f:
                    manifest_data = json.load(f)

                header = manifest_data.get('header', {})
                pack_id = header.get('uuid')
                version = header.get('version')
                name = header.get('name', 'Unknown')

                if not pack_id or not version:
                    continue

                print(f"   🔎 Ditemukan Pack: {name} ({os.path.basename(root)})")

                valid_packs_config.append({
                    "pack_id": pack_id,
                    "version": version,
                    "file_system": "RawPath",
                })

                if os.path.basename(root) == mod_folder_name_to_find:
                    print(f"   ✅ Ditemukan MOD KUSTOM! Mengaktifkan '{name}' untuk dunia.")
                    pack_to_activate = {"pack_id": pack_id, "version": version}
                    if pack_type == 'behavior' and pack_to_activate not in world_behavior_packs:
                        world_behavior_packs.append(pack_to_activate)
                    elif pack_type == 'resource' and pack_to_activate not in world_resource_packs:
                        world_resource_packs.append(pack_to_activate)

            except Exception as e:
                print(f"   ❌ Error membaca manifest di {manifest_path}: {e}")

process_packs(behavior_pack_folder, 'behavior')
process_packs(resource_pack_folder, 'resource')

if world_behavior_packs or world_resource_packs:
    print(f"\n📄 Membuat file konfigurasi...")

    with open(valid_packs_file, 'w') as f:
        json.dump(valid_packs_config, f, indent=2)
    print(f"   ✅ {valid_packs_file} dibuat dengan {len(valid_packs_config)} total pack(s) yang diketahui.")

    if world_behavior_packs:
        world_behavior_packs_file = os.path.join(world_folder, 'world_behavior_packs.json')
        with open(world_behavior_packs_file, 'w') as f:
            json.dump(world_behavior_packs, f, indent=2)
        print(f"   ✅ {world_behavior_packs_file} dibuat dengan {len(world_behavior_packs)} behavior pack(s) aktif.")

    if world_resource_packs:
        world_resource_packs_file = os.path.join(world_folder, 'world_resource_packs.json')
        with open(world_resource_packs_file, 'w') as f:
            json.dump(world_resource_packs, f, indent=2)
        print(f"   ✅ {world_resource_packs_file} dibuat dengan {len(world_resource_packs)} resource pack(s) aktif.")

    leveldat_path = os.path.join(world_folder, 'level.dat')
    if not os.path.exists(leveldat_path):
        with open(leveldat_path, 'wb') as f: f.write(b'')
        print(f"   ✅ File level.dat placeholder dibuat.")

    print(f"\n🎉 Konfigurasi dunia selesai!")
    print(f"   World '{world_name}' telah dikonfigurasi dengan mod 'Actions and Stuff' di penyimpanan sementara.")
else:
    print("\n❌ Tidak ada mod kustom yang terdeteksi atau dikonfigurasi!")
    print(f"   Pastikan ada folder bernama '{mod_folder_name_to_find}' di dalam 'behavior_packs' dan/atau 'resource_packs'.")

Mengkonfigurasi behavior pack dan resource pack untuk dunia...

📦 Mencari behavior packs...
   🔎 Ditemukan Pack: Experimental Creator Cameras (experimental_creator_cameras)
   🔎 Ditemukan Pack: resourcePack.vanilla_server.name (vanilla_1.18.30)
   🔎 Ditemukan Pack: resourcePack.vanilla_server.name (vanilla_1.21.20)
   🔎 Ditemukan Pack: resourcePack.vanilla_server.name (vanilla_1.20.80)
   🔎 Ditemukan Pack: resourcePack.vanilla_server.name (vanilla_1.17.40)
   🔎 Ditemukan Pack: resourcePack.vanilla_server.name (vanilla_1.17.20)
   🔎 Ditemukan Pack: resourcePack.vanilla_server.name (vanilla_1.21.50)
   ❌ Error membaca manifest di /content/minecraft/behavior_packs/chemistry_1.21.10/manifest.json: Expecting property name enclosed in double quotes: line 20 column 7 (char 436)
   🔎 Ditemukan Pack: resourcePack.vanilla_server.name (vanilla_1.19.0)
   🔎 Ditemukan Pack: resourcePack.vanilla_server.name (vanilla)
   🔎 Ditemukan Pack: resourcePack.vanilla_server.name (vanilla_1.19.80)
   🔎 Ditemu

In [5]:
server_properties_content = f"""
server-name=NewWorld Server with Mods
gamemode=survival
force-gamemode=false
difficulty=normal
allow-cheats=true
max-players=6
online-mode=true
allow-list=false
server-port=19133
server-portv6=19133
enable-lan-visibility=true
view-distance=32
tick-distance=4
player-idle-timeout=30
max-threads=8
level-name={world_name}
level-seed=-3899835130120818196
default-player-permission-level=member
texturepack-required=false
content-log-file-enabled=true
compression-threshold=1
compression-algorithm=zlib
server-authoritative-movement=server-auth
server-authoritative-block-breaking=true
player-movement-score-threshold=20
player-movement-action-direction-threshold=0.85
player-movement-distance-threshold=0.3
player-movement-duration-threshold-in-ms=500
correct-player-movement=false
server-authoritative-block-breaking-pick-range-scalar=1.5
chat-restriction=None
disable-player-interaction=false
client-side-chunk-generation-enabled=true
block-network-ids-are-hashes=true
disable-persona=false
disable-custom-skins=false
server-build-radius-ratio=Disabled
allow-outbound-script-debugging=false
allow-inbound-script-debugging=false
script-debugger-auto-attach=disabled
script-debugger-auto-listen=false
script-debugger-generate-documentation=false
"""

with open("/content/minecraft/server.properties", "w") as f:
    f.write(server_properties_content)

print("✅ File server.properties berhasil dibuat dan dikonfigurasi di penyimpanan sementara.")
print("\n--- KONFIGURASI PENTING ---")
print("1. ✅ Nama server: 'NewWorld Server with Mods'")
print("2. ✅ World name: 'NewWorld' (akan diunggah ke Google Drive)")
print("3. ✅ Cheats enabled: untuk command admin dan debugging")
print("4. ✅ Content logging: aktif untuk debugging pack")
print("5. ✅ Server authoritative movement: untuk kompatibilitas mod")
print("6. ⚠️  Semua file akan diunggah ke /content/drive/MyDrive/minecraft-server di langkah akhir")

✅ File server.properties berhasil dibuat dan dikonfigurasi di penyimpanan sementara.

--- KONFIGURASI PENTING ---
1. ✅ Nama server: 'NewWorld Server with Mods'
2. ✅ World name: 'NewWorld' (akan diunggah ke Google Drive)
3. ✅ Cheats enabled: untuk command admin dan debugging
4. ✅ Content logging: aktif untuk debugging pack
5. ✅ Server authoritative movement: untuk kompatibilitas mod
6. ⚠️  Semua file akan diunggah ke /content/drive/MyDrive/minecraft-server di langkah akhir


In [6]:
print("Mengkonfigurasi port server untuk dunia Minecraft...")

properties_file = '/content/minecraft/server.properties'
default_port = "19133"

if os.path.exists(properties_file):
    try:
        with open(properties_file, 'r') as f:
            lines = f.readlines()

        new_lines = []
        port_updated = False
        for line in lines:
            if line.strip().startswith('server-port='):
                new_line = f'server-port={default_port}\n'
                new_lines.append(new_line)
                print(f"   -> Mengatur port ke default: '{new_line.strip()}'")
                port_updated = True
            else:
                new_lines.append(line)

        if not port_updated:
            print("   ⚠️ Baris 'server-port=' tidak ditemukan. Menambahkan di akhir file.")
            new_lines.append(f'\nserver-port={default_port}\n')

        with open(properties_file, 'w') as f:
            f.writelines(new_lines)

        print(f"   ✅ Port server diatur ke: {default_port}")
    except Exception as e:
        print(f"   ❌ Gagal mengubah file server.properties: {e}")
else:
    print(f"   ❌ ERROR: File '{properties_file}' tidak ditemukan.")

print("\n✅ Konfigurasi port selesai!")
print(f"💡 Gunakan port {default_port} saat menjalankan server Minecraft secara lokal.")


Mengkonfigurasi port server untuk dunia Minecraft...
   -> Mengatur port ke default: 'server-port=19133'
   ✅ Port server diatur ke: 19133

✅ Konfigurasi port selesai!
💡 Gunakan port 19133 saat menjalankan server Minecraft secara lokal.


In [7]:
print("Mengarsipkan dunia Minecraft ke file NewWorld.zip di penyimpanan sementara...")

source_dir = '/content/minecraft'
zip_filename = '/content/NewWorld.zip'

if os.path.exists(zip_filename):
    print(f"🧹 Menghapus file zip lama di {zip_filename}...")
    os.remove(zip_filename)

print(f"📦 Mengarsipkan semua file dan folder dari {source_dir} ke {zip_filename}...")
try:
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(source_dir):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, '/content')
                zipf.write(file_path, arcname)
    print(f"✅ File {zip_filename} berhasil dibuat.")
except Exception as e:
    print(f"❌ Gagal membuat file zip: {e}")
    print("Harap periksa ruang penyimpanan atau izin akses.")
    raise

print("\n🔍 Memverifikasi isi file zip:")
with zipfile.ZipFile(zip_filename, 'r') as zipf:
    zip_contents = zipf.namelist()
    print(f"✅ Total file dalam zip: {len(zip_contents)}")
    print("Contoh isi (maksimum 20 file):")
    for item in zip_contents[:20]:
        print(f"   - {item}")

key_files = [
    'minecraft/server.properties',
    'minecraft/valid_known_packs.json',
    'minecraft/worlds/NewWorld/level.dat'
]
print("\n🔍 Memeriksa keberadaan file kunci dalam zip:")
for key_file in key_files:
    if key_file in zip_contents:
        print(f"✅ {key_file} ada dalam zip.")
    else:
        print(f"❌ {key_file} tidak ditemukan dalam zip!")

print("\n✅ Dunia Minecraft berhasil dibuat dan diarsipkan!")
print(f"📍 File zip tersedia di: {zip_filename}")
print("💡 Anda dapat mendownload file 'NewWorld.zip' secara manual dari panel file Colab di /content.")
print("💡 Untuk mendownload, buka panel file di sebelah kiri Colab, temukan 'NewWorld.zip', klik kanan, dan pilih 'Download'.")
print("💡 Gunakan file zip ini untuk menjalankan server Minecraft Bedrock secara lokal dengan mod 'Actions and Stuff'.")

Mengarsipkan dunia Minecraft ke file NewWorld.zip di penyimpanan sementara...
📦 Mengarsipkan semua file dan folder dari /content/minecraft ke /content/NewWorld.zip...
✅ File /content/NewWorld.zip berhasil dibuat.

🔍 Memverifikasi isi file zip:
✅ Total file dalam zip: 36898
Contoh isi (maksimum 20 file):
   - minecraft/bedrock_server
   - minecraft/allowlist.json
   - minecraft/permissions.json
   - minecraft/valid_known_packs.json
   - minecraft/profanity_filter.wlist
   - minecraft/release-notes.txt
   - minecraft/bedrock_server_how_to.html
   - minecraft/server.properties
   - minecraft/config/default/permissions.json
   - minecraft/definitions/attachables/iron_helmet.json
   - minecraft/definitions/attachables/iron_chestplate.player.json
   - minecraft/definitions/attachables/diamond_boots.json
   - minecraft/definitions/attachables/chainmail_helmet.json
   - minecraft/definitions/attachables/chainmail_leggings.player.json
   - minecraft/definitions/attachables/netherite_boots.playe